In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
import json

In [2]:
# Application Config
global EnableColorStream, EnableInfraredStream, EnableDepthStream
EnableColorStream = 0
ColorStreamResWidth = 1920
ColorStreamResHeight = 1080
ColorStreamFPS = 30

EnableInfraredStream = 1
InfraredStreamResWidth = 1280
InfraredStreamResHeight = 720
InfraredStreamFPS = 30

EnableDepthStream = 1
DepthStreamResWidth = 1280
DepthStreamResHeight = 720
DepthStreamFPS = 30

maxDistance = 1000      

In [3]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream:
    config.enable_stream(
        rs.stream.color, ColorStreamResWidth, ColorStreamResHeight, rs.format.bgr8, ColorStreamFPS)

if EnableInfraredStream:
    config.enable_stream(
        rs.stream.infrared, InfraredStreamResWidth, InfraredStreamResHeight, rs.format.bgr8, InfraredStreamFPS)
    
if EnableDepthStream:
    config.enable_stream(
        rs.stream.depth, DepthStreamResWidth, DepthStreamResHeight, rs.format.z16, DepthStreamFPS)

# Start streaming
cfg = pipeline.start(config)

# Alignment
if EnableColorStream:
    align_to = rs.stream.color
elif EnableInfraredStream:
    align_to = rs.stream.infrared

align = rs.align(align_to)
    
# Advanced settings
dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 4)
depth_sensor.set_option(rs.option.enable_auto_exposure, 1)

# Get depth scale
scale = depth_sensor.get_depth_scale()
print("depth scale:" + str(scale))

# PointCloud settings
pc = rs.pointcloud()

depth scale:0.0010000000474974513


In [4]:
# colorize depth image
def colorizeImage(input_image):
    minNum, maxNum, minLoc, maxLoc = cv2.minMaxLoc(input_image)

    if maxNum > maxDistance:
        maxNum = maxDistance
    
    scaleAlpha = 255 / maxNum
    input_image = cv2.convertScaleAbs(input_image, None, scaleAlpha, 0)
    depth_colormap = cv2.applyColorMap(input_image, cv2.COLORMAP_JET)
    
    return depth_colormap

In [5]:
# screen capture function
def procScreenCapture(caputre):  
    if EnableColorStream:
        fileName = "../imageCapture/color_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
        cv2.imwrite(fileName, color_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

    if EnableInfraredStream:
        fileName = "../imageCapture/infrared_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
        cv2.imwrite(fileName, infrared_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])

    if EnableDepthStream:
        fileName = "../imageCapture/depth_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
        cv2.imwrite(fileName, depth_colormap, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])


In [6]:
# mouse click function


In [7]:
# application flags
capture = 0
point = 0

while True:
    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    
    # Aligh frames
    aligned_frames = align.proccess(frames)
    
    if EnableColorStream:
        color_frame = aligned_frames.get_color_frame()
        if not color_frame:
            continue
            
    if EnableInfraredStream:
        infrared_frame = aligned_frames.first(rs.stream.infrared)
        if not infrared_frame:
            continue
    
    if EnableDepthStream:
        depth_frame = aligned_frames.get_depth_frame()
        if not depth_frame:
            continue
    
    # Convert images to numpy arrays
    if EnableColorStream:
        color_image = np.asanyarray(color_frame.get_data())
    
    if EnableInfraredStream:
        infrared_image = np.asanyarray(infrared_frame.get_data())
    
    if EnableDepthStream:
        depth_image = np.asanyarray(depth_frame.get_data())
    
    # colorize depth image
    depth_colormap = colorizeImage(depth_image)
        
    # Show images
    if EnableColorStream:
        cv2.imshow('Color image', color_image)
        #cv2.setMouseCallback('Color image', procMouseClick)
        
    if EnableInfraredStream:
        cv2.imshow('Infrared image', infrared_image)
        #cv2.setMouseCallback('Infrared image', procMouseClick)
    
    if EnableDepthStream:
        cv2.imshow('Depth image', depth_colormap)
        #cv2.setMouseCallback('Depth image', procMouseClick)
    
    # Screen capture
    if capture:
        procScreenCapture()
        capture = 0
        
    # Read device temperature
    '''
    temp = depth_sensor.get_option(rs.option.projector_temperature)
    print("proj: " + str(temp))
    temp = depth_sensor.get_option(rs.option.asic_temperature)
    print("asic: " + str(temp))
    '''
    
    # Generate point cloud
    if point is 1:
        point = 0
        points = pc.calculate(depth_frame)
        
        if EnableColorStream:
            pc.map_to(color_frame)
            
        if EnableInfraredStream:
            pc.map_to(infrared_frame)
            
        vertices = points.get_vertices()
        tex_coords = points.get_texture_coordinates()
        
        for i in range(0, points.size()):
            if vertices[i].z:
                print(vertices[i].x)
                print(vertices[i].y)
                print(vertices[i].z)
                break
        

    # Keyboard command
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c') or getKey is ord('C'):
        capture = 1
    elif getKey is ord('q') or getKey is ord('Q'):
        break
    elif getKey is ord('p') or getKey is ord('P'):
        point = 1

UnboundLocalError: local variable 'capture' referenced before assignment

In [ ]:
# Stop streaming
cv2.destroyAllWindows()
pipeline.stop()